# Tokenization Examples
通过这个网站[Tiktokenizer](https://tiktokenizer.vercel.app/?encoder=gpt2)可以感受一下分词器是如何运作的。
```
<|im_start|>system<|im_sep|>You are a helpful assistant<|im_end|>
<|im_start|>user<|im_sep|>I am studying CS336<|im_end|><|im_start|>assistant<|im_sep|>
```
这一段内容是一个示例，是我们和人工智能之间的对话的系统格式，我发现一个有趣的现象，如果直接将上面这段内容复制粘贴到ChatGPT中，并且让它给你解释一下，它只会认为你是一个正在学习CS336的同学，但是如果在外面加一个引号然后引上这段内容，ChatGPT才能为你解释这段内容。这也正好验证了这是人类和人工智能之间的对话的例子。

通过这个例子观察到的**现象**：
- 在英文中一个单词前面的空格会和它一起组成一个token；
- 相同的单词出现在句首和句中，token不一定相同，例如"hello"是句首的token，" hello"是句中token（注意有个空格），这也就会导致最后得到的整数不一样；
- 一个数字不会被当作一个整体，例如2025可能被分为两组来理解（202、5）



In [2]:
%pip install tiktoken
import tiktoken

Note: you may need to restart the kernel to use updated packages.


In [5]:
tokenizer = tiktoken.get_encoding("gpt2")
text = "Hello, 🌍! 你好!"
token_ids = tokenizer.encode(text)
print("Token IDs:", token_ids)

decoded_text = tokenizer.decode(token_ids)
print("Decoded Text:", decoded_text)

Token IDs: [15496, 11, 12520, 234, 235, 0, 220, 19526, 254, 25001, 121, 0]
Decoded Text: Hello, 🌍! 你好!


**compression ratio**是压缩率，指的是原始的字符串长度（以UTF-8编码）与分词后的token数量的比值。
- 比值越大代表每个token能代表的字节越多，也就是压缩效果好；
- 比值越小代表分词更细碎，压缩效果差。

In [9]:
def get_compression_ratio(string: str, indices: list[int]) -> float:
    """Given `string` that has been tokenized into `indices`, ."""
    num_bytes = len(bytes(string, encoding="utf-8"))  # @inspect num_bytes
    num_tokens = len(indices)                       # @inspect num_tokens
    return num_bytes / num_tokens
get_compression_ratio(text, token_ids)

1.6666666666666667

# Character-based tokenization
字符级分词器也就是将每一个字符都作为一个token。python中的字符串都是Unicode字符组成的序列，通过`ord("a")`可以获取字符对应的Unicode编码，反过来使用`chr(97)`可以获得对应的字符：

In [11]:
print(ord("a"))
print(chr(97))

97
a


In [15]:
# 我们可以通过这个映射关系得到一个字符级别的tokenizer
class CharacterTokenizer():
    def encode(self, string:str) -> list[int]:
        return list(map(ord, string))
    def decode(self, indices:list[int]) -> str:
        return "".join(map(chr, indices))
tokenizer = CharacterTokenizer()
string = "Hello, 🌍! 你好!"
indices = tokenizer.encode(string)
print(indices)
reconstructed_string = tokenizer.decode(indices)
print(reconstructed_string)

[72, 101, 108, 108, 111, 44, 32, 127757, 33, 32, 20320, 22909, 33]
Hello, 🌍! 你好!


这样做是可行的，但是问题在于Unicode定义了超过十五万个字符，由于每个字符都有一个唯一的ID，这样就导致了词汇表有十几万个映射，需要的内存太大；并且由于Unicode中大部分字符出现的频率其实很小，因此这会导致浪费了很多空间存储使用频率极低的字符

In [17]:
get_compression_ratio(string, indices)

1.5384615384615385

# Byte-based Tokenization
不按照词或者字符来分，而是将字符串转换为UTF-8编码的字节序列，每一个字节就是一个token。每一个字节都是一个0-255的整数，UTF-8是一种变长的编码方式，对于英文只需要一个字节，但是对于emoji或者汉字则需要3-4个字节

In [18]:
bytes("a", encoding="utf-8") == b"a"
bytes("🌍", encoding="utf-8") == b"\xf0\x9f\x8c\x8d"

True

In [20]:
class ByteTokenizer():
    """Represent a string as a sequence of bytes."""
    def encode(self, string: str) -> list[int]:
        string_bytes = string.encode("utf-8")  # 将字符串编码为UTF-8字节流
        indices = list(map(int, string_bytes))  # 将每个字节取出来，转换为整数列表
        return indices
    def decode(self, indices: list[int]) -> str:
        string_bytes = bytes(indices)  # 将整数列表打包为bytes类型
        string = string_bytes.decode("utf-8")  # 还原为字符串
        return string
tokenizer = ByteTokenizer()
string = "Hello, 🌍! 你好!"
indices = tokenizer.encode(string)
print(indices)
reconstructed_string = tokenizer.decode(indices)
print(reconstructed_string)

[72, 101, 108, 108, 111, 44, 32, 240, 159, 140, 141, 33, 32, 228, 189, 160, 229, 165, 189, 33]
Hello, 🌍! 你好!


这样做的好处就是不需要提前训练词表，但是这样的方法将任何文本都拆分的太细了，由于Transformer的注意力机制是$\mathcal{O}(N^2)$，这会导致计算量过大，因为token太多，序列太长，并且这也会导致后续推理过程也很慢。

In [22]:
compression_ratio = get_compression_ratio(string, indices)
compression_ratio

1.0

# Word-based tokenization
把一句话按照词为单位来分割，每个词作为一个token

In [24]:
import regex
string = "I'll say supercalifragilisticexpialidocious!"
segments = regex.findall(r"\w+|.", string) # \w+表示连续匹配字母、数字或者下划线 |.表示匹配任意单个字符（保留标点）
print(segments)

# GPT2使用了一个更为严格的regex
GPT2_TOKENIZER_REGEX = \
    r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
segments = regex.findall(GPT2_TOKENIZER_REGEX, string)
print(segments)

['I', "'", 'll', ' ', 'say', ' ', 'supercalifragilisticexpialidocious', '!']
['I', "'ll", ' say', ' supercalifragilisticexpialidocious', '!']


需要对每个分词有一个对应的ID，和字符一样，单词数量很多，可能会导致词表很庞大；并且很多罕见词可能只出现过一次，模型很难学到有用的表示；并且对于词表中没有的词语，会被映射为UNK，会导致其失去原有的信息

# BPE(Byte Pair Encoding)
原始的 Byte Pair Encoding 是一种压缩算法，用于反复找出文本中出现最频繁的相邻字符对，并把它们合并成一个新的符号。这种方法既能表示常见的词语，也能分解未知的词。

`@dataclass`是python3.7引入的一个装饰器，用于简化类的定义，特别适用于数据对象的类，只需要声明字段，python就会自动引入一大堆常用的方法，比如说`__init__`, `__eq__`等
```python
class Token:
    def __init__(self, id, value):
        self.id = id
        self.value = value

    def __repr__(self):
        return f"Token(id={self.id}, value={self.value})"

    def __eq__(self, other):
        return self.id == other.id and self.value == other.value
```
```python
from dataclasses import dataclass

@dataclass
class Token:
    id: int
    value: str
```

In [34]:
from collections import defaultdict
from dataclasses import dataclass
# forzen=True表示这个类的实例是不可变的，就像元组一样，可以防止被更改
@dataclass(frozen=True) 
class BPETokenizerParams:
    vocab: dict[int, bytes] # 词表，将整数和对应的byte字节串对应起来
    merges: dict[tuple[int, int], int] # BPE训练过程中学到的合并法则，将一个token对(index1,index2)合并为一个重新分配的token ID(new_index)
                                       # 例如 merges[(72, 105)] = 256

def merge(indices:list[int], pair:tuple[int, int], new_index:int) -> list[int]:
    new_indices = []
    i = 0
    while i < len(indices):
        if i + 1 < len(indices) and indices[i] == pair[0] and indices[i] == pair[1]:
            new_indices.append(new_index)
            i += 2
        else:
            new_indices.append(indices[i])
            i += 1
    return new_indices


def train_bpe(string: str, num_merges: int) -> BPETokenizerParams:
    indices = list(map(int, string.encode("utf-8")))
    merges: dict[tuple[int, int], int] = {}
    vocab: dict[int, bytes] = {x: bytes([x]) for x in range(256)} # 初始化词表
    for i in range(num_merges):
        counts = defaultdict(int) # 创建一个defaultdict对象，如果访问到一个不存在的值，不会跑出KeyError，而是会将这个新键初始化为int()的返回值
        for index1, index2 in zip(indices, indices[1:]): # zip会将多个序列的对应元素打包为一个个元组
            counts[(index1, index2)] += 1 # 最后会存储字符串中每个相邻字节对出现的次数
        pair = max(counts, key=counts.get) # 选出频率最高的pair
        index1, index2 = pair
        new_index = 256 + i
        merges[(index1, index2)] = new_index
        vocab[new_index] = vocab[index1] + vocab[index2]
        indices = merge(indices, pair, new_index)
    return BPETokenizerParams(vocab, merges)

class BPETokenizer():
    def __init__(self, params:BPETokenizerParams):
        self.params = params
    def encode(self, string:str) -> list[int]:
        indices = list(map(int, string.encode("utf-8"))) 
        for pair, new_index in self.params.merges.items():
            indices = merge(indices, pair, new_index)
        return indices
    def decode(self, indices:list[int]) -> str:
        # map(self.params.vocab.get, indices) 将indices中的每一个ID映射为vocab中对应的byte
        # bytes_list = [self.params.vocab.get(index) for index in indices]
        bytes_list = list(map(self.params.vocab.get, indices)) 
        string = b"".join(bytes_list).decode("utf-8")
        return string

string = "the cat in the hat"
params = train_bpe(string, 3)
tokenizer = BPETokenizer(params)
indices = tokenizer.encode(string)
print(indices)
reconstructed_string = tokenizer.decode(indices)
print(reconstructed_string)  

[116, 104, 101, 32, 99, 97, 116, 32, 105, 110, 32, 116, 104, 101, 32, 104, 97, 116]
the cat in the hat


In [38]:
params.merges

{(116, 104): 258}

In [41]:
b = "A".encode("utf-8") 
print(b) # 虽然打印出来是b'A'但是实际上里面存的是一个0-255的整数
bytes([9]) # 创建了一个长度为1的字节串，必须传入的是一个可迭代的对象

b'A'


b'\t'

`str.encode("utf-8")`将字符串变为字节串
`bytes.decode("utf-8")`将字节串变为字符串

在Python 中，`b''` 表示一个字节串(bytes object)，其中存储的每个字符都是一个byte。`list(b"Hi")`实际上就是吧字符串转变为一个字节值列表